In [1]:
!pip install dash dash-bootstrap-components

In [2]:
!pip install fpdf

In [3]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import adapt, register_adapter, AsIs
from sqlalchemy import create_engine
import getpass
import pandas as pd
import numpy as np

import dash
from dash import dcc, html, Input, Output, State, MATCH, ALL
import dash_bootstrap_components as dbc
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from fpdf import FPDF
import os
import webbrowser
import numpy as np
import re
from shapely.geometry import Point
from pyproj import Transformer
from shapely import wkt
from functools import lru_cache
from datetime import datetime

In [ ]:
database = 'f24t03'
user = 'jsmm8'
password = getpass.getpass("Type password and hit enter: ")

Type password and hit enter: ········


In [ ]:
try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(dbname=database, user=user, password=password, host='pgsql')
    cursor = connection.cursor()

    # Execute a query to get all table names
    cursor.execute("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public';
    """)
    
    # Fetch all results
    tables = cursor.fetchall()

    # Print the table names
    print("Tables in the database:")
    for table in tables:
        print(table[0])

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Tables in the database:
ethnicity
year_built
springfield_fire
education_levels
springfield_middle_school
tenure_by_bedrooms
tenure_by_kitchen
tenure_by_plumbing
rooms
imputed_median_house_value
raster_columns
raster_overviews
springfield_police
education_categories
demographics_master
model_features_data
year_structure_built
springfield_early_childhood_school
mortgage
tenure_by_rooms
income_data_processed
race_data_processed
housing_stats_data
race_migration_data
merged_new_house_data
income_pct_change
bedrooms
distance_drive_times
educational_attainment
employment_status
springfield_neighborhood_service_areas
parcel_boundaries
springfield_final_dataset
geo_mobility
home_value
house_heating_fuel
household_income
housing_pct_change_stats_data
internet_subscription
income_to_poverty_ratio
median_home_value
median_household_income
median_rooms_by_tenure
occupancy_status
plumbing_facilities
poverty_status
mortgage_status
springfield_single_family_point
springfield_elementary_school
springf

In [ ]:
# NSA
try:
    conn = psycopg2.connect(database=database,
                            user=user,
                            host='pgsql',
                            password=password)
    print("I am able to connect to the database")
except Exception as e:
    print("I am unable to connect to the database:", e)
    raise

# Define the query to select all data from the ethnicity table
query = "SELECT * FROM springfield_neighborhood_service_areas;"

try:
    nsa = gpd.read_postgis(query, conn, geom_col='geom')
    print("Data extracted successfully!")
except Exception as e:
    print("Error extracting data:", e)
finally:
    # Close the connection
    conn.close()

# Display the DataFrame
print(nsa.shape)
nsa.head()

I am able to connect to the database
Data extracted successfully!
(38, 7)


,objectid_1,name,shapestare,shapestlen,shape_length,shape_area,geom
0,1,Grant Beach,3.107893e+07,22403.823703,6826.057182,2.885105e+06,"MULTIPOLYGON (((474089.118 4120469.519, 474088..."
1,2,Oak Grove,5.694040e+07,32119.505522,9786.311928,5.285924e+06,"MULTIPOLYGON (((480052.310 4114905.475, 480052..."
2,3,Midtown,2.839690e+07,21373.071458,6512.016504,2.636137e+06,"MULTIPOLYGON (((475419.924 4118741.896, 475052..."
3,4,Weller,2.558836e+07,20759.618765,6325.117077,2.375422e+06,"MULTIPOLYGON (((476778.203 4118815.866, 476776..."
4,5,Woodland Heights,2.922756e+07,23012.009407,7011.363483,2.713241e+06,"MULTIPOLYGON (((472934.565 4120381.619, 472883..."


In [7]:
nsa.crs

{'init': 'epsg:26915'}

In [8]:
nsa.columns

Index(['objectid_1', 'name', 'shapestare', 'shapestlen', 'shape_length',
       'shape_area', 'geom'],
      dtype='object')

In [ ]:
# Full data

# === Step 1: Connect to your database ===
conn = psycopg2.connect(database=database, user=user, host='pgsql', password=password)

# === Step 2: Load full table using pandas ===
query = "SELECT * FROM georealedge_data;"
df = pd.read_sql(query, conn)
conn.close()

# === Step 3: Identify all geometry columns ===
geometry_cols = [col for col in df.columns if 'geolocation' in col]

# === Step 4: Convert all geometry columns from WKB to Shapely Points ===
from shapely import wkb

def safe_parse(val):
    try:
        if isinstance(val, str):
            return wkb.loads(val, hex=True)
        elif isinstance(val, (bytes, memoryview)):
            return wkb.loads(val)
    except:
        return None

for col in geometry_cols:
    df[col] = df[col].apply(safe_parse)

# === Step 5: Wrap as GeoDataFrame using one geometry column ===
df_full = gpd.GeoDataFrame(df, geometry='home_geolocation', crs='EPSG:26915')

df_full.head()

,home_geolocation,block_group_id,address,objectid,zip5,nsa,bds_zone,zoning,drive_time_fire,drive_distance_fire,...,square_footage,lot_size,year_built,sinkhole_house_risk_rating,sinkhole_risk_label,has_sinkhole_impact,pct_fld_oc,has_floodplain_impact,floodplain_house_risk_rating,floodplain_risk_label
0,POINT (471174.694 4120899.099),1500000US290770033003,2511 W ATLANTIC ST,6720,65803,TOM WATKIN,A,R-SF,2.75,0.92,...,2358,10688.0,2019,0,No Risk,N,0.0,N,0,No Risk
1,POINT (472875.801 4120913.727),1500000US290770019001,2027 N JOHNSTON AVE,52747,65803,WOODLAND,A,R-SF,3.30,0.98,...,2947,19842.0,1971,0,No Risk,N,0.0,N,0,No Risk
2,POINT (473810.378 4120914.316),1500000US290770019002,2031 N LYON AVE,56844,65803,WOODLAND,BN,R-SF,1.89,0.54,...,3388,19634.0,1979,0,No Risk,N,0.0,N,0,No Risk
3,POINT (471548.433 4120914.286),1500000US290770033003,2014 N PARK AVE,38608,65803,TOM WATKIN,A,R-SF,3.17,1.14,...,2832,21306.0,2014,0,No Risk,N,0.0,N,0,No Risk
4,POINT (474571.749 4120917.463),1500000US290770055003,2040 N BENTON AVE,5218,65803,WOODLAND,BN,R-SF,3.22,0.92,...,2594,10017.0,1962,0,No Risk,N,0.0,N,0,No Risk


In [10]:
# Final Interface

import dash
from dash import dcc, html, Input, Output, State, MATCH, ALL
import dash_bootstrap_components as dbc
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from fpdf import FPDF
import os
import webbrowser
import numpy as np
import re
from shapely.geometry import Point
from pyproj import Transformer
from shapely import wkt
from functools import lru_cache
from datetime import datetime

# ---------------------- DATA LOADING FUNCTIONS ----------------------

@lru_cache(maxsize=None)
def load_county_boundaries():
    """Load and cache county boundaries"""
    gdf = gpd.read_file("tl_2023_29_bg/tl_2023_29_bg.shp").to_crs(epsg=4326)
    counties_gdf = gdf.dissolve(by='COUNTYFP')
    return counties_gdf

@lru_cache(maxsize=None)
def load_greene_county_blockgroups():
    """Load and cache Greene County blockgroups"""
    gdf = gpd.read_file("tl_2023_29_bg/tl_2023_29_bg.shp").to_crs(epsg=4326)
    greene_blockgroups = gdf[gdf['COUNTYFP'] == '077']
    return greene_blockgroups

@lru_cache(maxsize=None)
def load_nsa_boundaries():
    """Load and cache NSA boundaries"""
    try:
        df = nsa.copy()
        geometries = [wkt.loads(wkt_str) if isinstance(wkt_str, str) else None for wkt_str in df['geom']]
        gdf = gpd.GeoDataFrame(df, geometry=geometries, crs="EPSG:4326")  # Set crs here directly
        # Remove the set_crs line
        return gdf
    except Exception as e:
        print(f"Error loading NSA boundaries: {e}")
        return gpd.GeoDataFrame()

def parse_geolocation(geoloc_str):
    """Parse geolocation string into a Point object"""
    try:
        if pd.isna(geoloc_str) or not isinstance(geoloc_str, str):
            return None
        
        match = re.search(r'POINT \((\d+\.\d+) (\d+\.\d+)\)', geoloc_str)
        if match:
            x, y = float(match.group(1)), float(match.group(2))
            
            transformer = Transformer.from_crs("EPSG:26915", "EPSG:4326", always_xy=True)
            lon, lat = transformer.transform(x, y)
            
            return Point(lon, lat)
        else:
            return wkt.loads(geoloc_str)
    except Exception as e:
        print(f"Error parsing geometry: {e} for input: {geoloc_str[:30]}...")
        return None

@lru_cache(maxsize=None)
def load_interface_data():
#Load and cache interface data
    df = df_full.copy()
    df['home_geometry'] = df['home_geolocation'].apply(parse_geolocation)
    return df

def piecewise_score(distance, threshold):
    """Calculate score based on distance from threshold"""
    if distance <= threshold:
        return 1.00
    elif distance <= threshold + 1:
        return 0.85
    elif distance <= threshold + 2:
        return 0.70
    elif distance <= threshold + 3:
        return 0.55
    elif distance <= threshold + 4:
        return 0.40
    else:
        return 0.25

# ---------------------- MAP CREATION FUNCTIONS ----------------------

def create_county_map(county_boundaries):
    """Create initial county map"""
    fig = px.choropleth_mapbox(
        county_boundaries,
        geojson=county_boundaries.__geo_interface__,
        locations=county_boundaries.index,
        color_discrete_sequence=["lightgray"],
        mapbox_style="carto-positron",
        center={"lat": county_boundaries.geometry.centroid.y.mean(), "lon": county_boundaries.geometry.centroid.x.mean()},
        zoom=5.5,
        opacity=0.5
    )
    fig.update_layout(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        coloraxis_showscale=False
    )
    return fig

def create_springfield_map(greene_blockgroups, nsa_boundaries=None):
    """Create Springfield detail map with optional NSA boundaries"""
    fig = px.choropleth_mapbox(
        greene_blockgroups,
        geojson=greene_blockgroups.__geo_interface__,
        locations=greene_blockgroups.index,
        color_discrete_sequence=["lightblue"],
        opacity=0.3,
        mapbox_style="carto-positron",
        zoom=10,
        center={"lat": 37.2090, "lon": -93.2923}  # Springfield coordinates
    )
    fig.update_layout(
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        title="Springfield Block Groups",
        coloraxis_showscale=False
    )
    
    # Add NSA boundaries
    if nsa_boundaries is not None and not nsa_boundaries.empty:
        add_nsa_boundaries_to_map(fig, nsa_boundaries)
        fig.update_layout(title="Springfield: Block Groups and NSA Boundaries")
        
    return fig

def add_nsa_boundaries_to_map(fig, nsa_boundaries):
    """Add NSA boundaries to an existing map figure"""
    for i, row in nsa_boundaries.iterrows():
        if row.geometry is None:
            continue
            
        nsa_name = row['NAME'] if 'NAME' in row else f"NSA {i}"
        
        if row.geometry.geom_type == 'MultiPolygon':
            for polygon in row.geometry.geoms:
                x, y = polygon.exterior.xy
                fig.add_trace(go.Scattermapbox(
                    lon=list(x),
                    lat=list(y),
                    mode='lines',
                    line=dict(width=2, color='black'),
                    hoverinfo='text',
                    hovertext=nsa_name,
                    showlegend=False
                ))
        elif row.geometry.geom_type == 'Polygon':
            x, y = row.geometry.exterior.xy
            fig.add_trace(go.Scattermapbox(
                lon=list(x),
                lat=list(y),
                mode='lines',
                line=dict(width=2, color='black'),
                hoverinfo='text',
                hovertext=nsa_name,
                showlegend=False
            ))

def add_home_points_to_map(fig, filtered_df, color='orange', size=8, opacity=0.7, name='Available Homes'):
    """Add home points to an existing map figure"""
    if 'home_geometry' in filtered_df.columns:
        filtered_df = filtered_df.copy()
        filtered_df['lon'] = filtered_df['home_geometry'].apply(lambda point: point.x if point else None)
        filtered_df['lat'] = filtered_df['home_geometry'].apply(lambda point: point.y if point else None)
        
        homes_with_coords = filtered_df.dropna(subset=['lat', 'lon'])
        
        if not homes_with_coords.empty:
            home_scatter = go.Scattermapbox(
                lat=homes_with_coords['lat'],
                lon=homes_with_coords['lon'],
                mode='markers',
                marker=dict(
                    size=size,
                    color=color,
                    opacity=opacity
                ),
                text=homes_with_coords['address'],
                hoverinfo='text',
                name=name
            )
            
            fig.add_trace(home_scatter)
    
    return fig

# ---------------------- FILTERING FUNCTIONS ----------------------

def filter_homes_data(df, price_range, income_range, bedrooms, bathrooms, sqft_range):
    """Apply all filters to the dataset"""
    filtered_df = df.copy()
    
    # Apply price filter
    if 'house_price' in filtered_df.columns and price_range:
        filtered_df = filtered_df[
            (filtered_df['house_price'] >= price_range[0]) & 
            (filtered_df['house_price'] <= price_range[1])
        ]
    
    # Apply income filter
    if 'median_income' in filtered_df.columns and income_range:
        filtered_df = filtered_df[
            (filtered_df['median_income'] >= income_range[0]) & 
            (filtered_df['median_income'] <= income_range[1])
        ]
    
    # Apply home specification filters
    if 'bedrooms' in filtered_df.columns and bedrooms is not None:
        filtered_df = filtered_df[filtered_df['bedrooms'] == bedrooms]
        
    if 'bathrooms' in filtered_df.columns and bathrooms is not None:
        filtered_df = filtered_df[filtered_df['bathrooms'] == bathrooms]
        
    if 'square_footage' in filtered_df.columns and sqft_range:
        filtered_df = filtered_df[
            (filtered_df['square_footage'] >= sqft_range[0]) & 
            (filtered_df['square_footage'] <= sqft_range[1])
        ]
    
    return filtered_df

def get_poi_data_from_store(poi_store, poi_column_mapping):
    """Get POI data from the central store for scoring"""
    # Extract the selections from the store
    selections = poi_store.get('selections', {})
    valid_poi_data = []
    
    # Process each selected POI
    for poi_name, poi_data in selections.items():
        if poi_data.get('selected', False):
            column_name = poi_column_mapping.get(poi_name)
            importance = poi_data.get('importance')
            distance = poi_data.get('distance')
            
            # Only add valid POIs with all required data
            if column_name and importance is not None and distance is not None:
                valid_poi_data.append({
                    'name': poi_name,
                    'column': column_name,
                    'importance': importance,
                    'distance': distance
                })
    
    # Print debug information
    print("\n==== POI DATA FROM CENTRAL STORE ====")
    for i, poi in enumerate(valid_poi_data):
        print(f"{i+1}. {poi['name']}: Importance={poi['importance']}, Distance={poi['distance']}")
    print("=======================================\n")
    
    return valid_poi_data

def calculate_scores(filtered_df, valid_poi_data):
    """Calculate POI scores based on user preferences"""
    if not valid_poi_data:
        return filtered_df
    
    scored_df = filtered_df.copy()
    
    # Extract data for scoring
    selected_pois = [poi['column'] for poi in valid_poi_data]
    user_thresholds = {poi['column']: poi['distance'] for poi in valid_poi_data}
    user_importance = {poi['column']: poi['importance'] for poi in valid_poi_data}
    
    # Convert importance values to integers if they're strings
    user_importance = {k: int(v) if isinstance(v, str) else v for k, v in user_importance.items()}
    
    # Compute weights
    weights = {poi: val / sum(user_importance.values()) for poi, val in user_importance.items()}
    
    # Calculate individual scores for each POI
    for poi in selected_pois:
        score_column = f"{poi}_score"
        scored_df[score_column] = scored_df[poi].apply(
            lambda x: piecewise_score(x, user_thresholds[poi])
        )
    
    # Calculate the final weighted score
    scored_df["final_score"] = scored_df.apply(
        lambda row: sum(
            row[f"{poi}_score"] * weights[poi] for poi in selected_pois if f"{poi}_score" in row
        ),
        axis=1
    )
    
    # Sort by final score
    scored_df = scored_df.sort_values("final_score", ascending=False)
    
    return scored_df

# ---------------------- PDF GENERATION ----------------------

class PropertyReportPDF(FPDF):
    """PDF Generator for property reports"""
    def header(self):
        # Logo and title
        self.set_font('Arial', 'B', 18)
        self.set_text_color(41, 128, 185)  # Blue
        self.cell(0, 10, 'GeoRealEdge', 0, 1, 'C')
        self.set_font('Arial', '', 12)
        self.cell(0, 10, 'Property Recommendation Match Report', 0, 1, 'C')
        
        # Add filter information in the header
        self.set_font('Arial', 'I', 8)
        self.set_text_color(100, 100, 100)  # Gray
        
        if hasattr(self, 'filter_info'):
            filter_text = self.filter_info
            self.cell(0, 5, filter_text, 0, 1, 'R')
        
        # Line break
        self.ln(5)
        
    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        # Page number
        self.cell(0, 10, f'Page {self.page_no()}/{{nb}}', 0, 0, 'C')
    
    def set_filter_info(self, price_range, bedrooms, bathrooms, sqft_range):
        self.filter_info = f"Filters: ${price_range[0]:,}-${price_range[1]:,} | {bedrooms} bed | {bathrooms} bath | {sqft_range[0]:,}-{sqft_range[1]:,} sq ft"
        
    def add_property_card(self, property_data, poi_data):
        # Add a property card with details in row-wise format
        self.set_fill_color(240, 240, 240)  # Light gray background
        
        # Property header with address
        self.set_font('Arial', 'B', 14)
        self.set_text_color(52, 73, 94)  # Dark gray
        
        # Property address as header
        address = str(property_data.get('address', 'N/A'))
        self.multi_cell(0, 10, address, 1, 'L', True)
        self.ln(5)
        
        # Create two-column layout for property details
        col_width = self.w / 2 - 15
        line_height = 8
        
        # Basic property details - left column
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, "Property Details", 0, 1, 'L')
        self.ln(1)
        
        # First row - basic details
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Price:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, self.format_currency(property_data.get('house_price', 'N/A')), 0, 0, 'L')
        
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Year Built:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('year_built', 'N/A')), 0, 1, 'L')
        
        # Second row
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Bedrooms:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('bedrooms', 'N/A')), 0, 0, 'L')
        
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Bathrooms:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('bathrooms', 'N/A')), 0, 1, 'L')
        
        # Third row
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Square Feet:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('square_footage', 'N/A')), 0, 0, 'L')
        
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "NSA:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('nsa', 'N/A')), 0, 1, 'L')
        
        self.ln(5)
        
        # Neighborhood details
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, "Neighborhood Details", 0, 1, 'L')
        self.ln(1)
        
        # Neighborhood demographics
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Median Income:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, self.format_currency(property_data.get('median_income', 'N/A')), 0, 0, 'L')
        
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Race:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('race', 'N/A')), 0, 1, 'L')
        
        # Second row
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Diversity:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('diversity', 'N/A')), 0, 0, 'L')
        
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Education:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('education_score_category', 'N/A')), 0, 1, 'L')
        
        # Third row
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Sink Hole Impact:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('sinkhole_house_risk_rating', 'N/A')), 0, 0, 'L')
        
        self.set_font('Arial', 'B', 10)
        self.cell(40, line_height, "Flood Plain Impact:", 0, 0, 'L')
        self.set_font('Arial', '', 10)
        self.cell(col_width - 40, line_height, str(property_data.get('floodplain_house_risk_rating', 'N/A')), 0, 1, 'L')
        self.ln(5)
        
        # Points of Interest
        if poi_data:
            self.set_font('Arial', 'B', 12)
            self.cell(0, 10, "Points of Interest (Distance in Miles)", 0, 1, 'L')
            self.ln(1)
            
            # Create a table for POI data
            # Headers
            self.set_font('Arial', 'B', 10)
            self.set_fill_color(230, 230, 230)
            self.cell(80, line_height, "Location", 1, 0, 'L', True)
            self.cell(30, line_height, "Distance (mi)", 1, 0, 'C', True)
            self.cell(30, line_height, "Drive Time", 1, 0, 'C', True)
            self.cell(30, line_height, "Importance", 1, 1, 'C', True)
            
            # Data rows
            self.set_font('Arial', '', 10)
            self.set_fill_color(255, 255, 255)
            for poi in poi_data:
                # Get the distance column and value
                distance_val = property_data.get(poi['column'], 'N/A')
                distance_str = f"{distance_val:.1f}" if isinstance(distance_val, (int, float)) else str(distance_val)
                
                # Get the corresponding drive time column and value
                time_column = poi['column'].replace('distance', 'time')
                time_val = property_data.get(time_column, 'N/A')
                time_str = f"{time_val:.1f} min" if isinstance(time_val, (int, float)) else str(time_val)
                
                self.cell(80, line_height, poi['name'], 1, 0, 'L')
                self.cell(30, line_height, distance_str, 1, 0, 'C')
                self.cell(30, line_height, time_str, 1, 0, 'C')
                self.cell(30, line_height, str(poi['importance']), 1, 1, 'C')
        
        # Match score
        self.ln(5)
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, "Match Score", 0, 1, 'L')
        self.ln(1)
        
        final_score = property_data.get('final_score', 'N/A')
        if isinstance(final_score, (int, float)):
            score_percentage = final_score * 100
            score_str = f"{score_percentage:.2f}%"
            # Add visual indicator
            self.set_fill_color(150, 200, 150)  # Light green
            y_position = self.get_y()
            self.rect(10, y_position, self.w - 20, 10, 'F')
            # Score text
            self.set_font('Arial', 'B', 14)
            self.set_text_color(0, 100, 0)  # Dark green
            self.cell(0, 10, f"Score: {score_str}", 0, 1, 'C')
            self.set_text_color(0, 0, 0)  # Reset to black
        else:
            self.set_font('Arial', '', 10)
            self.cell(0, 8, f"Score: {str(final_score)}", 0, 1, 'L')
        
        # Add page break after each property
        self.add_page()
        
    def format_currency(self, value):
        if isinstance(value, (int, float)):
            return f"${value:,.2f}"
        return str(value)

    def add_appendix(self, poi_data, county, price_range, income_range, filter_values):
        """Add an appendix section to the PDF with additional information including risk ratings"""
        # Create appendix section header
        self.add_page()
        self.set_font('Arial', 'B', 18)
        self.set_text_color(41, 128, 185)  # Blue
        self.cell(0, 15, 'Appendix', 0, 1, 'C')
        self.ln(5)

        # Risk Rating Explanations
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, 'Environmental Risk Ratings', 0, 1, 'L')
        self.ln(5)
        
        # Sinkhole Risk Rating
        self.set_font('Arial', 'B', 12)
        self.cell(0, 8, 'Sinkhole Risk Rating:', 0, 1, 'L')
        self.ln(2)
        
        # Table header for sinkhole
        self.set_font('Arial', 'B', 10)
        self.set_fill_color(230, 230, 230)
        self.cell(20, 8, "Rating", 1, 0, 'C', True)
        self.cell(40, 8, "Risk Level", 1, 0, 'C', True)
        self.cell(130, 8, "Description", 1, 1, 'C', True)
        
        # Table data for sinkhole
        self.set_font('Arial', '', 10)
        sinkhole_ratings = [
            ["0", "No Risk", "0% parcel over sinkhole"],
            ["1", "Very Low", "<10% of parcel over sinkhole or houses with sinkholes nearby (<=10m)"],
            ["2", "Low", "10-25% parcel over sinkhole"],
            ["3", "Moderate", "25-50% parcel over sinkhole"],
            ["4", "High", "50-90% parcel over sinkhole"],
            ["5", "Severe", "100% parcel over sinkhole"]
        ]
        
        for rating in sinkhole_ratings:
            self.cell(20, 8, rating[0], 1, 0, 'C')
            self.cell(40, 8, rating[1], 1, 0, 'C')
            self.cell(130, 8, rating[2], 1, 1, 'L')
        
        self.ln(10)
        
        # Floodplain Risk Rating
        self.set_font('Arial', 'B', 12)
        self.cell(0, 8, 'Floodplain Risk Rating:', 0, 1, 'L')
        self.ln(2)
        
        # Table header for floodplain
        self.set_font('Arial', 'B', 10)
        self.set_fill_color(230, 230, 230)
        self.cell(20, 8, "Rating", 1, 0, 'C', True)
        self.cell(40, 8, "Risk Level", 1, 0, 'C', True)
        self.cell(130, 8, "Description", 1, 1, 'C', True)
        
        # Table data for floodplain
        self.set_font('Arial', '', 10)
        floodplain_ratings = [
            ["1", "Very Low", "<10% covered by floodplain"],
            ["2", "Low", "10-25% covered by floodplain"],
            ["3", "Moderate", "25-50% covered by floodplain"],
            ["4", "High", "50-90% covered by floodplain"],
            ["5", "Severe", "100% covered by floodplain"]
        ]
        
        for rating in floodplain_ratings:
            self.cell(20, 8, rating[0], 1, 0, 'C')
            self.cell(40, 8, rating[1], 1, 0, 'C')
            self.cell(130, 8, rating[2], 1, 1, 'L')

        self.ln(10)
        
        # Educational Attainment Rating
        self.set_font('Arial', 'B', 12)
        self.set_text_color(41, 128, 185)  # Blue to match header
        self.cell(0, 8, 'Educational Attainment Rating:', 0, 1, 'L')
        self.ln(2)
        
        # Table header for educational attainment
        self.set_font('Arial', 'B', 10)
        self.set_text_color(0, 0, 0)  # Reset text color to black
        self.set_fill_color(230, 230, 230)
        self.cell(40, 8, "Rating Level", 1, 0, 'C', True)
        self.cell(150, 8, "Description", 1, 1, 'C', True)
        
        # Table data for educational attainment with multi-line cells
        self.set_font('Arial', '', 10)
        education_ratings = [
            ["Very Low", "Indicates areas with the lowest weighted average educational attainment, likely dominated by populations with little or no formal education."],
            ["Low", "Represents slightly higher educational levels than \"Very Low\", but still largely skewed toward lower education such as high school or below."],
            ["Medium", "Marks moderate educational attainment, where the population likely has a mix of high school graduates and some college experience."],
            ["High", "Suggests a greater share of individuals with associate's or bachelor's degrees, showing upward educational mobility."],
            ["Very High", "Reflects the highest levels of educational attainment, dominated by bachelor's and advanced degree holders."]
        ]
        
        for rating in education_ratings:
            self.cell(40, 10, rating[0], 1, 0, 'C')
            x_position = self.get_x()
            y_position = self.get_y()
            self.multi_cell(150, 10, rating[1], 1, 'L')
            # Reset position for next row
            self.set_xy(self.l_margin, self.get_y())
            
        self.ln(5)
        self.cell(0, 5, "2025 GeoRealEdge", 0, 1, 'C')

def generate_property_pdf(filtered_df, valid_poi_data, price_range, income_range, filter_values, county):
    """Generate a PDF report for perfect score properties"""
    try:
        # Get perfect score homes
        perfect_score_df = filtered_df[filtered_df['final_score'] >= 0.98].copy()
        if perfect_score_df.empty:
            return False, "No perfect scores found to include in PDF."
        
        # Create the PDF
        pdf = PropertyReportPDF()
        pdf.alias_nb_pages()
        pdf.add_page()
        
        # Add filter information to header
        bedrooms = filter_values.get('bedrooms', 0)
        bathrooms = filter_values.get('bathrooms', 0)
        sqft_range = filter_values.get('sqft_range', [1000, 1000])
        pdf.set_filter_info(price_range, bedrooms, bathrooms, sqft_range)
        
        # Add summary page
        pdf.set_font('Arial', 'B', 16)
        pdf.cell(0, 10, 'Property Recommendation Match Report', 0, 1, 'C')
        
        # Add date and filters information
        pdf.set_font('Arial', '', 10)
        pdf.cell(0, 10, f'Generated on: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', 0, 1, 'R')
        pdf.cell(0, 10, f'County: {county}', 0, 1, 'L')
        
        # Add detailed filter criteria
        pdf.cell(0, 6, f'Price Range: ${price_range[0]:,} - ${price_range[1]:,}', 0, 1, 'L')
        pdf.cell(0, 6, f'Income Range: ${income_range[0]:,} - ${income_range[1]:,}', 0, 1, 'L')
        pdf.cell(0, 6, f'Bedrooms: {bedrooms}', 0, 1, 'L')
        pdf.cell(0, 6, f'Bathrooms: {bathrooms}', 0, 1, 'L')
        pdf.cell(0, 6, f'Square Footage: {sqft_range[0]:,} - {sqft_range[1]:,} sq ft', 0, 1, 'L')
        
        # POI Selection summary
        pdf.ln(5)
        pdf.set_font('Arial', 'B', 12)
        pdf.cell(0, 10, 'Points of Interest Selected:', 0, 1, 'L')
        pdf.set_font('Arial', '', 10)
        
        for poi in valid_poi_data:
            pdf.cell(0, 8, f"- {poi['name']} (Importance: {poi['importance']}, Max Distance: {poi['distance']} miles)", 0, 1, 'L')
        
        # Add list of properties
        pdf.ln(10)
        pdf.set_font('Arial', 'B', 12)
        pdf.cell(0, 10, f"Properties Matched: {len(perfect_score_df)}", 0, 1, 'L')

        for i, (_, home) in enumerate(perfect_score_df.iterrows()):
            address = str(home.get('address', f"Property {i+1}"))
            pdf.set_font('Arial', '', 10)
            pdf.cell(0, 8, f"{i+1}. {address}", 0, 1, 'L')
        
        # Add property details pages
        for _, home in perfect_score_df.iterrows():
            pdf.add_property_card(home, valid_poi_data)
        pdf.add_appendix(valid_poi_data, county, price_range, income_range, filter_values)
        
        # Save the PDF
        pdf_path = "perfect_score_homes.pdf"
        pdf.output(pdf_path)
        
        return True, "PDF generated successfully!"
        
    except Exception as e:
        print(f"Error generating PDF: {e}")
        return False, f"Error generating PDF: {str(e)}"

# ---------------------- UI COMPONENTS ----------------------

def create_stats_cards(locations_df, perfect_score_count):
    """Create statistics cards for the results dashboard"""
    median_sqft = locations_df['square_footage'].median() if 'square_footage' in locations_df.columns else 0
    
    return dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.Div([
                        html.I(className="fas fa-home fa-2x text-primary me-3"),
                        html.Div([
                            html.H3(f"{len(locations_df)}", className="mb-0 text-primary"),
                            html.P("Total Matching Homes", className="mb-0 text-muted small")
                        ])
                    ], className="d-flex align-items-center")
                ])
            ], className="shadow-sm h-100 border-0")
        ], width=12, md=4, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.Div([
                        html.I(className="fas fa-award fa-2x text-success me-3"),
                        html.Div([
                            html.H3(f"{perfect_score_count}", className="mb-0 text-success"),
                            html.P("Perfect Score Homes", className="mb-0 text-muted small")
                        ])
                    ], className="d-flex align-items-center")
                ])
            ], className="shadow-sm h-100 border-0")
        ], width=12, md=4, className="mb-3"),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.Div([
                        html.I(className="fas fa-ruler-combined fa-2x text-info me-3"),
                        html.Div([
                            html.H3(f"{median_sqft:.0f} sqft", className="mb-0 text-info"),
                            html.P("Median Size", className="mb-0 text-muted small")
                        ])
                    ], className="d-flex align-items-center")
                ])
            ], className="shadow-sm h-100 border-0")
        ], width=12, md=4, className="mb-3"),
    ], className="mb-4")

def create_property_card(home):
    """Create a property card for an individual home"""
    score = home.get('final_score', 0)
    score_percentage = int(score * 100)
    
    # Determine the color based on score
    if score >= 0.98:
        score_color = "success"
        score_text_class = "text-success"
    elif score >= 0.8:
        score_color = "info"
        score_text_class = "text-info"
    elif score >= 0.6:
        score_color = "primary"
        score_text_class = "text-primary"
    else:
        score_color = "secondary"
        score_text_class = "text-secondary"
    
    # Create a property card
    card = dbc.Card([
        dbc.Row([
            # Score display
            dbc.Col([
                html.Div([
                    html.H2(f"{score_percentage}%", className=f"{score_text_class} mb-0 text-center"),
                    html.P("Match", className="text-center mb-0 small text-muted")
                ], className="d-flex flex-column justify-content-center align-items-center h-100")
            ], width=2, className="border-end"),
            
            # Property details
            dbc.Col([
                html.H5(home.get('address', 'Unknown Address'), className="card-title mb-2"),
                dbc.Row([
                    dbc.Col([
                        html.Span(f"{home.get('bedrooms', 'N/A')} ", className="fw-bold"),
                        html.Span("bed", className="text-muted"),
                    ], width="auto", className="pe-2"),
                    dbc.Col([
                        html.Span(f"{home.get('bathrooms', 'N/A')} ", className="fw-bold"),
                        html.Span("bath", className="text-muted"),
                    ], width="auto", className="pe-2"),
                    dbc.Col([
                        html.Span(f"{int(home.get('square_footage', 0)):,d} ", className="fw-bold"),
                        html.Span("sq ft", className="text-muted"),
                    ], width="auto", className="pe-2"),
                    dbc.Col([
                        html.Span(f"${int(home.get('house_price', 0)):,d}", className="fw-bold text-success"),
                    ], width="auto"),
                ], className="mb-2"),
                html.P(f"NSA: {home.get('nsa', 'N/A')}", className="mb-0 small text-muted")
            ], width=10)
        ], className="g-0")
    ], className="mb-3 shadow-sm border-0")
    
    return card

def create_results_dashboard(locations_df, price_range, income_range, bedrooms, bathrooms, sqft_range):
    """Create the complete results dashboard"""
    # Calculate count of perfect score homes
    perfect_score_count = len(locations_df[locations_df['final_score'] >= 0.98])
    
    # Create stats cards
    stats_cards = create_stats_cards(locations_df, perfect_score_count)
    
    # Create a card for each top result
    top_results = locations_df.head(10)
    result_cards = [create_property_card(home) for _, home in top_results.iterrows()]
    
    # Create filter summary card
    filter_summary = dbc.Card([
        dbc.CardHeader([
            html.H5([
                html.I(className="fas fa-filter me-2"),
                "Applied Filters"
            ], className="mb-0 small")
        ]),
        dbc.CardBody([
            dbc.Row([
                dbc.Col([
                    html.Span("Price: ", className="fw-bold"),
                    html.Span(f"${price_range[0]:,} - ${price_range[1]:,}")
                ], width=12, md=6, className="mb-2"),
                dbc.Col([
                    html.Span("Income: ", className="fw-bold"),
                    html.Span(f"${income_range[0]:,} - ${income_range[1]:,}")
                ], width=12, md=6, className="mb-2"),
                dbc.Col([
                    html.Span("Specs: ", className="fw-bold"),
                    html.Span(f"{bedrooms} bed, {bathrooms} bath, {sqft_range[0]:,}-{sqft_range[1]:,} sq ft")
                ], width=12, className="mb-2"),
            ]),
        ], className="py-2")
    ], className="mb-4 shadow-sm border-0")
    
    # Main results card with property listings
    properties_card = dbc.Card([
        dbc.CardHeader([
            dbc.Row([
                dbc.Col([
                    html.H5([
                        html.I(className="fas fa-list me-2"),
                        "Top Matching Properties"
                    ], className="mb-0")
                ], width=8),
                dbc.Col([
                    dbc.Button([
                        html.I(className="fas fa-file-pdf me-2"),
                        "Download Perfect Score Results"
                    ], id="download-pdf-button", color="success", className="float-end")
                ], width=4, className="text-end")
            ])
        ]),
        dbc.CardBody([
            html.Div(id="pdf-status"),
            html.Div(result_cards)
        ])
    ], className="shadow-sm border-0")
    
    # Create data table columns and data
    display_columns = ["address", "bedrooms", "bathrooms", "square_footage", "final_score"]
    column_labels = {
        "address": "Address",
        "bedrooms": "Bedrooms",
        "bathrooms": "Bathrooms",
        "square_footage": "Square Feet",
        "final_score": "Match Score"
    }
    
    # Complete dashboard layout
    return dbc.Card([
        dbc.CardHeader([
            html.H4([
                html.I(className="fas fa-chart-line me-2"),
                "Results Dashboard"
            ], className="text-primary")
        ]),
        dbc.CardBody([
            html.P(f"Properties matching your criteria", className="text-muted mb-4"),
            
            stats_cards,
            
            filter_summary,
            
            properties_card,
            
            html.Div([
                dbc.Button([
                    html.I(className="fas fa-table me-2"),
                    "View All Results in Table Format"
                ], id="toggle-table-view", color="primary", outline=True, className="mt-3 w-100")
            ]),
            
            # Hidden table view (initially)
            dbc.Collapse([
                dash.dash_table.DataTable(
                    id='results-table',
                    columns=[{"name": column_labels.get(col, col), "id": col} for col in display_columns],
                    data=locations_df[display_columns].round(2).to_dict('records'),
                    sort_action='native',
                    filter_action='native',
                    page_size=10,
                    style_table={
                        'overflowX': 'auto',
                        'border': '1px solid #e0e0e0',
                        'borderRadius': '8px',
                        'boxShadow': '0 2px 6px rgba(0, 0, 0, 0.1)',
                    },
                    style_header={
                        'backgroundColor': '#f8f9fa',
                        'fontWeight': 'bold',
                        'textAlign': 'center',
                        'border': '1px solid #dee2e6',
                        'padding': '12px 15px'
                    },
                    style_cell={
                        'padding': '10px 15px',
                        'textAlign': 'left',
                        'fontFamily': 'Arial, sans-serif',
                    },
                    style_data={
                        'border': '1px solid #f0f0f0'
                    },
                    style_data_conditional=[
                        # Highlight perfect score rows
                        {
                            'if': {'filter_query': '{final_score} >= 0.98'},
                            'backgroundColor': '#e8f4f8',
                            'fontWeight': 'bold',
                            'border': '2px solid #4CAF50'
                        },
                        # High scores
                        {
                            'if': {'filter_query': '{final_score} >= 0.8 && {final_score} < 0.98'},
                            'backgroundColor': '#edfbef',
                            'border': '1px solid #a8e6a8'
                        },
                        # Medium scores
                        {
                            'if': {'filter_query': '{final_score} >= 0.6 && {final_score} < 0.8'},
                            'backgroundColor': '#fff9e6',
                        },
                        # Style score column
                        {
                            'if': {'column_id': 'final_score'},
                            'backgroundImage': 'linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%)',
                            'fontWeight': 'bold',
                            'textAlign': 'center'
                        },
                        # Style other columns
                        {
                            'if': {'column_id': 'address'},
                            'width': '40%'
                        },
                        {
                            'if': {'column_id': 'bedrooms'},
                            'textAlign': 'center'
                        },
                        {
                            'if': {'column_id': 'bathrooms'},
                            'textAlign': 'center'
                        },
                        {
                            'if': {'column_id': 'square_footage'},
                            'textAlign': 'center'
                        }
                    ]
                )
            ], id="table-view-container", is_open=False, className="mt-3")
        ])
    ], className="shadow-sm border-0")

def create_price_income_filters(price_min, price_max, income_min, income_max):
    """Create price and income range filter components"""
    return html.Div([
        dbc.Card([
            dbc.CardHeader([
                html.H5([
                    html.I(className="fas fa-dollar-sign me-2 text-success"),
                    "Financial Filters"
                ], className="mb-0")
            ]),
            dbc.CardBody([
                html.Label('💵 Price Range:', className="fw-bold text-dark fs-4 d-flex align-items-center slider-label"),
                html.Div([
                    dcc.RangeSlider(
                        id='price-range-slider',
                        min=price_min,
                        max=price_max,
                        step=10000,
                        marks={i: f'${i//1000}k' for i in range(int(price_min), int(price_max) + 1, int((price_max - price_min) / 5))},
                        value=[price_min, price_min],
                        className="my-3 bigger-slider lg-slider"
                    ),
                    html.Div(id="price-range-display", className="mt-1 fw-bold text-dark fs-4 text-center value-display")
                ], className="slider-container"),
                
                html.Label('💰 Income Range:', className="fw-bold text-dark fs-4 d-flex align-items-center slider-label mt-4"),
                html.Div([
                    dcc.RangeSlider(
                        id='income-range-slider',
                        min=income_min,
                        max=income_max,
                        step=5000,
                        marks={i: f'${i//1000}k' for i in range(int(income_min), int(income_max) + 1, int((income_max - income_min) / 5))},
                        value=[income_min, income_min],
                        className="my-3 bigger-slider lg-slider"
                    ),
                    html.Div(id="income-range-display", className="mt-1 fw-bold text-dark fs-4 text-center value-display")
                ], className="slider-container"),
            ])
        ], className="shadow-sm mb-4 border-0"),
    ])

def create_home_filters():
    """Create home specification filter components"""
    return html.Div([
        dbc.Card([
            dbc.CardHeader([
                html.H5([
                    html.I(className="fas fa-home me-2 text-info"),
                    "Property Specifications"
                ], className="mb-0")
            ]),
            dbc.CardBody([
                html.Label('🛏️ Bedrooms:', className="fw-bold text-dark fs-4 d-flex align-items-center slider-label"),
                html.Div([
                    dcc.Slider(
                        id='bedroom-slider',
                        min=0,
                        max=6,
                        step=1,
                        marks={i: str(i) for i in range(0, 7)},
                        value=0,
                        className="my-3 bigger-slider lg-slider"
                    ),
                ], className="slider-container"),
                
                html.Label('🚿 Bathrooms:', className="fw-bold text-dark fs-4 d-flex align-items-center slider-label mt-4"),
                html.Div([
                    dcc.Slider(
                        id='bathroom-slider',
                        min=0,
                        max=5,
                        step=0.5,
                        marks={i: str(i) for i in range(0, 6)},
                        value=0,
                        className="my-3 bigger-slider lg-slider"
                    ),
                ], className="slider-container"),
                
                html.Label('📏 Square Footage:', className="fw-bold text-dark fs-4 d-flex align-items-center slider-label mt-4"),
                html.Div([
                    dcc.RangeSlider(
                        id='sqft-slider',
                        min=1000,
                        max=4500,
                        step=100,
                        marks={i: f'{i}' for i in range(1000, 4501, 500)},
                        value=[1000, 1000],
                        className="my-3 bigger-slider lg-slider"
                    ),
                    html.Div(id="sqft-range-display", className="mt-1 fw-bold text-dark fs-4 text-center value-display")
                ], className="slider-container"),
            ])
        ], className="shadow-sm mb-4 border-0"),
    ])

def create_poi_accordion(poi_options):
    """Create POI selection accordion with centralized state management"""
    poi_elements = []
    for i, poi in enumerate(poi_options):
        poi_elements.append(
            dbc.AccordionItem(
                [
                    dbc.Row([
                        dbc.Col([
                            dbc.Switch(
                                id={'type': 'poi-checkbox', 'index': poi},
                                value=False,
                                label=poi,
                                className="custom-switch"
                            )
                        ], width=12, className="mb-3"),
                        
                        dbc.Col([
                            html.Div(id={'type': 'poi-input-container', 'index': poi}, className="mb-2"),
                            # Add hidden div to track state
                            html.Div(id={'type': 'poi-state-tracker', 'index': poi}, style={'display': 'none'})
                        ], width=12)
                    ])
                ],
                title=[
                    html.I(className="fas fa-map-marker-alt me-2"),
                    poi
                ],
                item_id=f"item-{i}"
            )
        )
    
    return html.Div([
        dbc.Card([
            dbc.CardHeader([
                html.H5([
                    html.I(className="fas fa-map-pin me-2 text-danger"),
                    "Points of Interest"
                ], className="mb-0")
            ]),
            dbc.CardBody([
                html.P([
                    html.I(className="fas fa-info-circle me-2"),
                    "Select locations that matter to you, set importance (1-5) and preferred distance (miles)"
                ], className="text-muted mb-3"),
                dbc.Accordion(
                    poi_elements,
                    start_collapsed=True,
                    flush=True,
                    id="poi-accordion"
                )
            ])
        ], className="shadow-sm mb-4 border-0"),
    ])

# ---------------------- APPLICATION SETUP ----------------------

def initialize_app():
    """Initialize the application data and maps"""
    # Load data
    county_boundaries = load_county_boundaries()
    greene_blockgroups = load_greene_county_blockgroups()
    nsa_boundaries = load_nsa_boundaries()
    interface_df = load_interface_data()
    counties = ["Greene County"]
    
    # Get price and income ranges from the data
    price_min = interface_df['house_price'].min() if 'house_price' in interface_df.columns else 100000
    price_max = interface_df['house_price'].max() if 'house_price' in interface_df.columns else 500000
    income_min = interface_df['median_income'].min() if 'median_income' in interface_df.columns else 20000
    income_max = interface_df['median_income'].max() if 'median_income' in interface_df.columns else 150000
    
    # Create initial maps
    initial_fig = create_county_map(county_boundaries)
    initial_springfield_fig = create_springfield_map(greene_blockgroups, nsa_boundaries)
    
    # Points of Interest with mapping to actual column names
    poi_options = ["Hospital", "High School", "Early Childhood School", "Elementary School", 
                  "Middle School", "trailheads", "Urgent Care", "Fire Department", 
                  "Police", "Nursing Home", "Public health"]
    
    # Mapping from display names to actual column names in the CSV
    poi_column_mapping = {
        "Hospital": "drive_distance_hospital",
        "High School": "drive_distance_high_school",
        "Early Childhood School": "drive_distance_early_childhood_school",
        "Elementary School": "drive_distance_elementary_school",
        "Middle School": "drive_distance_middle_school",
        "trailheads": "drive_distance_trailheads",
        "Urgent Care": "drive_distance_urgentcare", 
        "Fire Department": "drive_distance_fire",
        "Police": "drive_distance_police",
        "Nursing Home": "drive_distance_nursing_home",
        "Public health": "drive_distance_public_health"
    }
    
    return {
        'county_boundaries': county_boundaries,
        'greene_blockgroups': greene_blockgroups,
        'nsa_boundaries': nsa_boundaries,
        'interface_df': interface_df,
        'counties': counties,
        'price_min': price_min,
        'price_max': price_max,
        'income_min': income_min,
        'income_max': income_max,
        'initial_fig': initial_fig,
        'initial_springfield_fig': initial_springfield_fig,
        'poi_options': poi_options,
        'poi_column_mapping': poi_column_mapping
    }

# Main Application Layout
def create_app_layout(poi_options, price_min, price_max, income_min, income_max, counties, initial_fig, initial_springfield_fig):
    """Create the main application layout with central POI store"""
    return dbc.Container([
        # Navigation Bar
        dbc.Navbar(
            dbc.Container([
                html.A(
                    dbc.Row([
                        dbc.Col(html.I(className="fas fa-map-marked-alt fa-2x text-primary"), width="auto"),
                        dbc.Col(dbc.NavbarBrand("GEO REAL EDGE", className="ms-2 fs-2 fw-bold text-primary")),
                    ],
                    align="center",
                    ),
                    href="#",
                    style={"textDecoration": "none"},
                ),
                dbc.NavbarToggler(id="navbar-toggler"),
                dbc.Collapse(
                    dbc.Nav([
                        dbc.NavItem(dbc.NavLink([html.I(className="fas fa-home me-1"), "Dashboard"], href="#", active=True)),
                    ], className="ms-auto", navbar=True),
                    id="navbar-collapse",
                    navbar=True,
                ),
            ]),
            color="white",
            light=True,
            className="mb-4 shadow-sm border-bottom",
            style={"padding": "0.75rem 0"},
        ),

        # Main Content
        dbc.Row([
            # Left Sidebar for Filters
            dbc.Col([
                dbc.Card([
                    dbc.CardHeader([
                        html.H4([
                            html.I(className="fas fa-sliders-h me-2 text-primary"),
                            "Search Parameters"
                        ], className="mb-0")
                    ]),
                    dbc.CardBody([
                        html.Div([
                            html.Label('📍 Select County:', className="fw-bold mb-2"),
                            dcc.Dropdown(
                                id='county-dropdown',
                                options=[{'label': county, 'value': county} for county in counties],
                                value=None,
                                placeholder="Select a county",
                                className="mb-4"
                            ),
                            html.Button([
                                html.I(className="fas fa-search me-2"),
                                "Search Homes"
                            ], id="initial-search-button", n_clicks=0, 
                                        className="btn btn-primary w-100 mb-4"),
                                        
                            # Available homes count display (initially hidden)
                            html.Div(id="home-count-display", className="d-none my-3"),
                                        
                            # Filters container (initially hidden, will be shown after selecting county)
                            html.Div(id='filters-container', className="mt-3"),
                        ])
                    ]),
                ], className="shadow-sm mb-4 border-0"),
                
                # Information Card
                dbc.Card([
                    dbc.CardHeader([
                        html.H5([
                            html.I(className="fas fa-info-circle me-2 text-info"),
                            "About NSA Boundaries"
                        ], className="mb-0")
                    ]),
                    dbc.CardBody([
                        html.P("Boundaries on the map represent NSA regions in City of Springfield, Missouri."),
                    ]),
                ], className="shadow-sm d-none d-lg-block border-0"),
            ], width=12, lg=4, className="mb-4"),
            
            # Main Content Area
            dbc.Col([
                # Top section with maps
                dbc.Card([
                    dbc.CardHeader([
                        html.H4([
                            html.I(className="fas fa-map me-2 text-primary"),
                            "Geographic Visualization"
                        ], className="mb-0")
                    ]),
                    dbc.CardBody([
                        dbc.Tabs([
                            dbc.Tab([
                                dcc.Graph(id='map', figure=initial_fig, style={"height": "60vh"})
                            ], label="NSA Overview", tab_id="tab-1"),
                            dbc.Tab([
                                dcc.Graph(id='springfield-map', figure=initial_springfield_fig, style={"height": "60vh"})
                            ], label="Springfield Detail", tab_id="tab-2"),
                        ], id="map-tabs", active_tab="tab-1"),
                    ]),
                ], className="shadow-sm mb-4 border-0"),
                
                # Results section
                html.Div(id='scoring-results-container', className="mb-4"),
            ], width=12, lg=8),
        ]),
        
        # Footer
        dbc.Row([
            dbc.Col([
                html.Hr(),
                html.Div([
                    html.Span("© 2025 ", className="text-muted"),
                    html.Span("GeoRealEdge", className="fw-bold text-primary"),
                    html.Span(" - Real Estate Data Analytics Platform", className="text-muted")
                ], className="text-center")
            ], width=12),
        ]),
        
        html.Div([
            # Store for filter values
            dcc.Store(id='filter-values', data={'bedrooms': 3, 'bathrooms': 2, 'sqft_range': [1500, 3000]}),
            
            dcc.Store(id='poi-central-store', data={
                'selections': {},
            }),
            
            html.Div(id='poi-state-reset', style={'display': 'none'})
        ], style={'display': 'none'}),
    ], fluid=True, className="px-4 py-3")

# ---------------------- DASH APP INITIALIZATION ----------------------

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX, 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css'], suppress_callback_exceptions=True)
app.title = "Geo Real Edge"

# Add custom CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .custom-switch .form-check-input {
                height: 1.25rem;
                width: 2.5rem;
            }
            .form-check-input:checked {
                background-color: #4CAF50;
                border-color: #4CAF50;
            }
            .card {
                border-radius: 10px;
                overflow: hidden;
            }
            .card-header {
                background-color: #fcfcfc;
                border-bottom: 1px solid #f0f0f0;
            }
            .btn-primary {
                background-color: #2196F3;
                border-color: #2196F3;
            }
            .btn-primary:hover {
                background-color: #0d8bf2;
                border-color: #0d8bf2;
            }
            .btn-success {
                background-color: #4CAF50;
                border-color: #4CAF50;
            }
            .btn-success:hover {
                background-color: #3d8b40;
                border-color: #3d8b40;
            }
            /* Improved slider styles */
            .bigger-slider .rc-slider-track {
                height: 8px; /* Increased from 6px */
            }
            .bigger-slider .rc-slider-rail {
                height: 8px; /* Increased from 6px */
            }
            .bigger-slider .rc-slider-handle {
                width: 24px; /* Increased from 20px */
                height: 24px; /* Increased from 20px */
                margin-top: -8px; /* Adjusted for larger handle */
                border-color: #2196F3;
            }
            /* Add hover effect to slider handles */
            .bigger-slider .rc-slider-handle:hover {
                border-color: #0d8bf2;
                box-shadow: 0 0 0 5px rgba(33, 150, 243, 0.2);
            }
            /* Larger text for slider marks */
            .bigger-slider .rc-slider-mark-text {
                font-size: 16px;  /* Increased from 14px */
                font-weight: 600; /* Made bolder */
                color: #333;      /* Darker color for better visibility */
            }
            
            /* Decrease importance dropdown size */
            .importance-dropdown {
                font-size: 0.9rem;
                padding: 4px 8px;
                height: auto;
            }
            
            /* Increase distance slider size */
            .distance-slider .rc-slider-track {
                height: 10px;
            }
            .distance-slider .rc-slider-rail {
                height: 10px;
            }
            .distance-slider .rc-slider-handle {
                width: 26px;
                height: 26px;
                margin-top: -8px;
            }
            
            /* Larger values for display */
            .value-display {
                font-size: 18px;  /* Increased from 16px */
                font-weight: 600;
            }
            
            /* Larger numbers for distance */
            .distance-value {
                font-size: 16px;
                font-weight: 600; 
            }
            
            /* Media query for large screens */
            @media (min-width: 1200px) {
                .slider-container {
                    padding: 0 10px;
                }
                /* Even bigger sliders on large screens */
                .lg-slider .rc-slider-track {
                    height: 10px;
                }
                .lg-slider .rc-slider-rail {
                    height: 10px;
                }
                .lg-slider .rc-slider-handle {
                    width: 28px;
                    height: 28px;
                    margin-top: -9px;
                }
                /* Increased font size for slider labels on large screens */
                .slider-label {
                    font-size: 16px;
                    margin-bottom: 8px;
                }
                /* Make large screen sliders even bigger */
                .bigger-slider .rc-slider-mark-text {
                    font-size: 18px;
                }
                .value-display {
                    font-size: 20px;
                }
                .distance-slider .rc-slider-track {
                    height: 12px;
                }
                .distance-slider .rc-slider-rail {
                    height: 12px;
                }
                .distance-slider .rc-slider-handle {
                    width: 30px;
                    height: 30px;
                    margin-top: -9px;
                }
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# Initialize application data
app_data = initialize_app()

# Set up application layout
app.layout = create_app_layout(
    app_data['poi_options'],
    app_data['price_min'],
    app_data['price_max'],
    app_data['income_min'],
    app_data['income_max'],
    app_data['counties'],
    app_data['initial_fig'],
    app_data['initial_springfield_fig']
)

# ---------------------- POI STORE CALLBACKS ----------------------

# Callback to update the POI central store when POI checkbox status changes
@app.callback(
    Output('poi-central-store', 'data'),
    [Input({'type': 'poi-checkbox', 'index': ALL}, 'value'),
     Input({'type': 'poi-importance', 'index': ALL}, 'value'),
     Input({'type': 'poi-distance', 'index': ALL}, 'value')],
    [State({'type': 'poi-checkbox', 'index': ALL}, 'id'),
     State({'type': 'poi-importance', 'index': ALL}, 'id'),
     State({'type': 'poi-distance', 'index': ALL}, 'id'),
     State('poi-central-store', 'data')],
    prevent_initial_call=True
)
def update_poi_central_store(
    checkbox_values, importance_values, distance_values, 
    checkbox_ids, importance_ids, distance_ids, current_store
):
    """
    Update the central POI store whenever any POI selection, importance, or distance changes.
    This is the key function for maintaining consistent POI state across multiple rounds of changes.
    """
    selections = current_store.get('selections', {})
    
    ctx = dash.callback_context
    if not ctx.triggered:
        return current_store
    
    # Handle checkbox changes - these determine if a POI is selected
    for i, (checkbox_id, value) in enumerate(zip(checkbox_ids, checkbox_values)):
        poi_name = checkbox_id['index']
        
        # Initialize or update this POI's data
        if poi_name not in selections:
            selections[poi_name] = {
                'selected': value,
                'importance': 3,  # Default values
                'distance': 5
            }
        else:
            selections[poi_name]['selected'] = value
    
    # Handle importance value changes
    for i, (importance_id, value) in enumerate(zip(importance_ids, importance_values)):
        if value is not None:
            poi_name = importance_id['index']
            if poi_name in selections:
                selections[poi_name]['importance'] = value
    
    # Handle distance value changes
    for i, (distance_id, value) in enumerate(zip(distance_ids, distance_values)):
        if value is not None:
            poi_name = distance_id['index']
            if poi_name in selections:
                selections[poi_name]['distance'] = value
    
    # Update the central store
    current_store['selections'] = selections
    
    # Debug output
    print("\n==== POI CENTRAL STORE UPDATED ====")
    print(f"Total POIs in store: {len(selections)}")
    selected_count = sum(1 for poi_data in selections.values() if poi_data.get('selected', False))
    print(f"Selected POIs: {selected_count}")
    for poi_name, poi_data in selections.items():
        if poi_data.get('selected', False):
            print(f"  - {poi_name}: Importance={poi_data.get('importance')}, Distance={poi_data.get('distance')}")
    print("====================================\n")
    
    return current_store

@app.callback(
    Output('poi-central-store', 'data', allow_duplicate=True),
    Input('initial-search-button', 'n_clicks'),
    State('poi-central-store', 'data'),
    prevent_initial_call=True
)
def reset_poi_store(n_clicks, current_store):
    """Reset the POI central store when the initial search button is clicked"""
    if not n_clicks:
        return dash.no_update
        
    return {'selections': {}}

@app.callback(
    Output({'type': 'poi-input-container', 'index': MATCH}, 'children'),
    [Input({'type': 'poi-checkbox', 'index': MATCH}, 'value'),
     Input('poi-central-store', 'data')],
    State({'type': 'poi-checkbox', 'index': MATCH}, 'id'),
    prevent_initial_call=True
)
def display_poi_inputs(selected_poi, poi_store, poi_id):
    """Display input controls for a selected POI, using values from the central store"""
    if not selected_poi:
        return ""
    
    poi_name = poi_id['index']
    
    # Retrieve stored values for this POI from the central store
    poi_data = poi_store.get('selections', {}).get(poi_name, {})
    importance_value = poi_data.get('importance', 3)
    distance_value = poi_data.get('distance', 5)

    return html.Div([
        dbc.Row([
            dbc.Col([
                html.Label("Importance:", className="fw-bold slider-label"),
                dbc.Select(
                    id={'type': 'poi-importance', 'index': poi_name},
                    options=[
                        {"label": "1 - Very Low", "value": 1},
                        {"label": "2 - Low", "value": 2},
                        {"label": "3 - Moderate", "value": 3},
                        {"label": "4 - High", "value": 4},
                        {"label": "5 - Very High", "value": 5}
                    ],
                    value=importance_value,
                    className="mb-3 importance-dropdown"
                )
            ], width=12, md=4),

            dbc.Col([
                html.Label("Preferred Distance (miles):", className="fw-bold slider-label"),
                html.Div([
                    dcc.Slider(
                        id={'type': 'poi-distance', 'index': poi_name},
                        min=0, max=10, step=0.5,
                        value=distance_value,
                        marks={i: str(i) for i in range(11)},
                        className="mb-3 bigger-slider lg-slider distance-slider"
                    ),
                    html.Div(
                        id={'type': 'poi-distance-display', 'index': poi_name},
                        children=f"{distance_value} miles",
                        className="mt-1 distance-value text-center"
                    )
                ], className="slider-container")
            ], width=12, md=8)
        ])
    ])

# ---------------------- CALLBACKS ----------------------

# Callback to update the filter values store
@app.callback(
    Output('filter-values', 'data'),
    [Input('bedroom-slider', 'value'),
     Input('bathroom-slider', 'value'),
     Input('sqft-slider', 'value')],
    State('filter-values', 'data')
)
def update_filter_values(bedrooms, bathrooms, sqft_range, current_values):
    if bedrooms is not None:
        current_values['bedrooms'] = bedrooms
    if bathrooms is not None:
        current_values['bathrooms'] = bathrooms
    if sqft_range is not None:
        current_values['sqft_range'] = sqft_range
    return current_values

# Callback to display homes count
@app.callback(
    [Output('home-count-display', 'children'),
     Output('home-count-display', 'className')],
    [Input('price-range-slider', 'value'),
     Input('income-range-slider', 'value'),
     Input('bedroom-slider', 'value'),
     Input('bathroom-slider', 'value'),
     Input('sqft-slider', 'value')],
    [State('county-dropdown', 'value')]
)
def update_home_count(price_range, income_range, bedrooms, bathrooms, sqft_range, county):
    if not county or not all([price_range, income_range, bedrooms is not None, bathrooms is not None, sqft_range]):
        return "", "d-none"
    
    # Filter data based on all current criteria
    filtered_df = filter_homes_data(
        app_data['interface_df'], 
        price_range, 
        income_range, 
        bedrooms, 
        bathrooms, 
        sqft_range
    )
    
    # Get count of remaining homes
    home_count = len(filtered_df)
    
    # Create the display element with appropriate styling
    if home_count > 0:
        count_display = dbc.Alert(
            [
                html.H4(f"📊 {home_count} homes", className="alert-heading mb-0"), 
                html.Span("match your current filter criteria", className="ms-2")
            ],
            color="success", 
            className="d-flex align-items-center mb-3",
            style={"fontSize": "1.2rem"}
        )
        return count_display, "d-block"
    else:
        count_display = dbc.Alert(
            [
                html.H4("⚠️ 0 homes", className="alert-heading mb-0"), 
                html.Span("match your current filter criteria. Try adjusting your filters.", className="ms-2")
            ],
            color="warning", 
            className="d-flex align-items-center mb-3",
            style={"fontSize": "1.2rem"}
        )
        return count_display, "d-block"

# Callbacks for displaying selected range values
@app.callback(
    Output('price-range-display', 'children'),
    [Input('price-range-slider', 'value')]
)
def update_price_display(price_range):
    if not price_range:
        return ""
    return f"Selected Range: ${price_range[0]:,} - ${price_range[1]:,}"

@app.callback(
    Output('income-range-display', 'children'),
    [Input('income-range-slider', 'value')]
)
def update_income_display(income_range):
    if not income_range:
        return ""
    return f"Selected Range: ${income_range[0]:,} - ${income_range[1]:,}"

@app.callback(
    Output('sqft-range-display', 'children'),
    [Input('sqft-slider', 'value')]
)
def update_sqft_display(sqft_range):
    if not sqft_range:
        return ""
    return f"Selected Range: {sqft_range[0]:,} - {sqft_range[1]:,} sq ft"

# Callback to update distance display
@app.callback(
    Output({'type': 'poi-distance-display', 'index': MATCH}, 'children'),
    Input({'type': 'poi-distance', 'index': MATCH}, 'value')
)
def update_distance_display(value):
    if value is None:
        return ""
    return f"{value} miles"

# Enhanced filters container
@app.callback(
    Output('filters-container', 'children'),
    [Input('initial-search-button', 'n_clicks')],
    [State('county-dropdown', 'value')]
)
def display_filters_and_poi_options(n_clicks, county):
    if not n_clicks or not county:
        return ""
    
    if county == "Greene County":
        # Create price and income range sliders
        price_income_filters = create_price_income_filters(
            app_data['price_min'], 
            app_data['price_max'], 
            app_data['income_min'], 
            app_data['income_max']
        )
        
        # Create home filters
        home_filters = create_home_filters()
        
        # Create POI section
        poi_section = create_poi_accordion(app_data['poi_options'])
        
        # Add final search button with enhanced styling
        final_search_button = dbc.Card([
            dbc.CardBody([
                html.Button(
                    [
                        html.I(className="fas fa-filter me-2"),
                        "Apply Filters and Score Homes"
                    ],
                    id="submit-button", 
                    n_clicks=0, 
                    className="btn btn-lg btn-primary w-100"
                )
            ])
        ], className="shadow-sm mb-4 border-0")
        
        # Combine all filter sections
        return html.Div([
            price_income_filters,
            home_filters,
            poi_section,
            final_search_button
        ])
    
    return ""

@app.callback(
    Output("table-view-container", "is_open"),
    [Input("toggle-table-view", "n_clicks")],
    [State("table-view-container", "is_open")],
)
def toggle_table_view(n, is_open):
    if n:
        return not is_open
    return is_open

# MAIN CALLBACK: Filter data and update dashboard using the central POI store
@app.callback(
    [Output('map', 'figure'),
     Output('springfield-map', 'figure'),
     Output('scoring-results-container', 'children')],
    Input('submit-button', 'n_clicks'),
    [State('price-range-slider', 'value'),
     State('income-range-slider', 'value'),
     State('county-dropdown', 'value'),
     State('filter-values', 'data'),
     State('poi-central-store', 'data')],
    prevent_initial_call=True
)
def update_dashboard_and_filter_data(n_clicks, price_range, income_range, county, 
                                     filter_values, poi_store):
    """
    Main callback to update all visualizations based on filters and POI selections.
    Uses the central POI store instead of component properties.
    """
    if not n_clicks or not county:
        return app_data['initial_fig'], app_data['initial_springfield_fig'], ""
    
    try:
        # Extract values from filter_values
        bedrooms = filter_values.get('bedrooms', 0)
        bathrooms = filter_values.get('bathrooms', 0)
        sqft_range = filter_values.get('sqft_range', [1000, 1000])
        
        # Filter data
        filtered_df = filter_homes_data(
            app_data['interface_df'], 
            price_range, 
            income_range, 
            bedrooms, 
            bathrooms, 
            sqft_range
        )
        
        # Get POI data from the central store
        valid_poi_data = get_poi_data_from_store(poi_store, app_data['poi_column_mapping'])
        
        # Create NSA boundaries map
        if county == "Greene County" and not app_data['nsa_boundaries'].empty:
            # Create figure for NSA boundaries
            updated_fig = go.Figure()
            
            # Add NSA boundary traces
            add_nsa_boundaries_to_map(updated_fig, app_data['nsa_boundaries'])
        else:
            county_fips = {'Greene County': '077'}  
            selected_fips = county_fips.get(county)
            
            if selected_fips:
                county_gdf = app_data['county_boundaries'][app_data['county_boundaries'].index.str.startswith(selected_fips)]
                updated_fig = px.choropleth_mapbox(
                    county_gdf,
                    geojson=county_gdf.__geo_interface__,
                    locations=county_gdf.index,
                    color_discrete_sequence=["blue"],
                    mapbox_style="carto-positron",
                    center={"lat": county_gdf.geometry.centroid.y.mean(), "lon": county_gdf.geometry.centroid.x.mean()},
                    zoom=8,
                    opacity=0.3
                )
            else:
                updated_fig = app_data['initial_fig']
        
        # Add home points to the map
        add_home_points_to_map(updated_fig, filtered_df)
        
        updated_fig.update_layout(
            mapbox=dict(
                style="carto-positron",
                center={"lat": 37.2090, "lon": -93.2923},  # Center on Springfield
                zoom=9.5
            ),
            margin={"r": 0, "t": 0, "l": 0, "b": 0},
            height=500
        )
        
        # Initialize scoring results container
        scoring_results_container = ""
        
        # Create Springfield map with block groups
        springfield_fig = create_springfield_map(app_data['greene_blockgroups'], app_data['nsa_boundaries'])
        
        # Only proceed with POI scoring if we have valid POI data
        if valid_poi_data:
            # Calculate scores
            scored_df = calculate_scores(filtered_df, valid_poi_data)
            
            if scored_df.empty:
                scoring_results_container = html.Div([
                    html.I(className="fas fa-exclamation-triangle me-2"),
                    "No homes match your criteria. Try adjusting your filters."
                ], className="alert alert-warning")
                return updated_fig, springfield_fig, scoring_results_container
            
            # Create results dashboard
            scoring_results_container = create_results_dashboard(
                scored_df, 
                price_range, 
                income_range, 
                bedrooms, 
                bathrooms, 
                sqft_range
            )
            
            # If we have perfect score homes, add them to the map
            perfect_score_df = scored_df[scored_df['final_score'] >= 0.98].copy()
            if not perfect_score_df.empty:
                # Add perfect score homes to Springfield map
                add_home_points_to_map(
                    springfield_fig, 
                    perfect_score_df, 
                    color='green', 
                    size=8, 
                    opacity=1.0, 
                    name='Perfect Score Homes'
                )
                
                springfield_fig.update_layout(
                    title="Springfield: Block Groups and Perfect Score Homes",
                    legend=dict(
                        yanchor="top",
                        y=0.99,
                        xanchor="left",
                        x=0.01
                    )
                )
        else:
            # No POIs selected
            scoring_results_container = html.Div([
                html.I(className="fas fa-info-circle me-2"),
                "Please select at least one Point of Interest to calculate scores."
            ], className="alert alert-info")
        
        return updated_fig, springfield_fig, scoring_results_container
        
    except Exception as e:
        print(f"Error in dashboard update: {str(e)}")
        error_message = html.Div([
            html.I(className="fas fa-exclamation-triangle me-2"),
            f"An error occurred while processing your request: {str(e)}. Please try refreshing the page."
        ], className="alert alert-danger")
        return app_data['initial_fig'], app_data['initial_springfield_fig'], error_message

# PDF generation callback
@app.callback(
    Output('pdf-status', 'children'),
    Input('download-pdf-button', 'n_clicks'),
    [State('county-dropdown', 'value'),
     State('price-range-slider', 'value'),
     State('income-range-slider', 'value'),
     State('filter-values', 'data'),
     State('poi-central-store', 'data')],
    prevent_initial_call=True
)
def generate_pdf(n_clicks, county, price_range, income_range, filter_values, poi_store):
    """Generate PDF report using the central POI store"""
    if not n_clicks or not county:
        return html.Div([
            html.I(className="fas fa-exclamation-circle me-2"),
            "No data available for PDF download."
            ], className="text-danger mt-2")
    
    # Filter data
    filtered_df = filter_homes_data(
        app_data['interface_df'], 
        price_range, 
        income_range, 
        filter_values.get('bedrooms', 0), 
        filter_values.get('bathrooms', 0), 
        filter_values.get('sqft_range', [1000, 1000])
    )
    
    # Get POI data from the central store
    valid_poi_data = get_poi_data_from_store(poi_store, app_data['poi_column_mapping'])
    
    if not valid_poi_data:
        # If no POIs selected, we can't calculate scores
        return html.Div([
            html.I(className="fas fa-exclamation-triangle me-2"),
            "Please select at least one Point of Interest to generate scores."
        ], className="text-warning mt-2")
    
    # Calculate scores
    scored_df = calculate_scores(filtered_df, valid_poi_data)
    
    # Generate PDF
    success, message = generate_property_pdf(
        scored_df, 
        valid_poi_data, 
        price_range, 
        income_range, 
        filter_values, 
        county
    )
    
    if success:
        return html.Div([
            html.I(className="fas fa-check-circle me-2 text-success"),
            message
        ], className="alert alert-success mt-2")
    else:
        return html.Div([
            html.I(className="fas fa-exclamation-circle me-2"),
            message
        ], className="alert alert-danger mt-2")

# Add Navbar collapse callback
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

# Run the app
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8050/')
    app.run_server(debug=True)